In [15]:
!pip install pypdf
!pip install transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama_index
!pip install llama-index-llms-huggingface
!pip install langchain-community langchain-core

In [16]:
!pip install llama-index-embeddings-langchain


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from huggingface_hub import login
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:

# Load documents
documents = SimpleDirectoryReader("/content/").load_data()

#prompt
system_prompt = """
You are a healthcare Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
query_wrapper_prompt = SimpleInputPrompt("{query_str}")


In [5]:
login(token="hf_login") #HF login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}
)


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

#query engine
query_engine = index.as_query_engine()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-15df54d75515>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [8]:
response = query_engine.query("what is PCOD?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


PCOD stands for Polycystic Ovarian Disease. It is a hormonal disorder that affects ovulation and is characterized by the presence of multiple small cysts on the ovaries. Symptoms of PCOD include irregular menstrual cycles, weight gain, acne, and excessive hair growth on the face and body. PCOD is often associated with insulin resistance and can increase the risk of developing type 2 diabetes. Treatment options for PCOD include medications to regulate hormones and ovulation, as well as lifestyle changes such as a healthy diet and regular exercise.


In [17]:
!pip install gradio

In [10]:
import gradio as gr

def chatbot(query):
    response = query_engine.query(query)
    return str(response)

interface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="Women's Healthcare Q&A Chatbot",
    description="Ask questions related to healthcare."
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bbe5cfa5d071fc07c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
response = query_engine.query("how long does menstruation last?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Menstruation typically lasts between 3 to 7 days, with the average duration being around 5 days. However, it's important to note that the duration of menstruation can vary from person to person, and some individuals may experience longer or shorter periods than others.


In [12]:
response = query_engine.query("how long does menstrual cycle last?")
print(response)

A typical menstrual cycle lasts around 28 days, but it can range from 21 to 35 days.


In [13]:
response = query_engine.query("what are menstraul taboos?")
print(response)


Menstrual taboos are social and cultural beliefs and practices that restrict or regulate menstrual practices, often perpetuating stigma and limiting access to resources and information. These taboos can vary widely across cultures and regions, and may include beliefs about menstruation being impure or unclean, or restrictions on menstruating individuals' participation in social and cultural activities. Menstrual taboos can also limit access to menstrual products and healthcare services, and may perpetuate gender inequality and discrimination. By recognizing and respecting diverse beliefs and practices surrounding menstruation, cultural competence and sensitivity can improve menstrual health interventions and promote menstrual equity.


In [14]:
response = query_engine.query("explain menstrual hygiene and how to maintain it?")
print(response)


Menstrual hygiene refers to the practices and habits that help maintain cleanliness and hygiene during menstruation. Proper menstrual hygiene practices are essential to prevent infections, maintain confidence, and feel comfortable during menstruation. Here are some tips to maintain good menstrual hygiene:

1. Use appropriate menstrual products: Choose the right menstrual product for your needs, such as pads, tampons, or menstrual cups. Use the product according to the manufacturer's instructions and change it regularly.
2. Change your menstrual product every 4-8 hours: Depending on your flow, change your menstrual product every 4-8 hours to prevent leakage and odor.
3. Wash your hands before and after handling menstrual products: Wash your hands with soap and water before and after handling menstrual products to prevent bacterial growth and infections.
4. Clean your body regularly: Clean your genital area with warm water and mild soap every 4-6 hours during menstruation. This helps re